$\textbf{Optimal Exercise of American Options}$

For European Call/Put Option:

The Black-Scholes equation is:
$C(S,t)=S_t\Phi\left(d_1\right)-e^{-r(T-t)}K\Phi\left(d_2\right),$

$d_1=\frac{\log(\frac{S_t}{K})+(r+\frac{\sigma^2}/{2})(T-t)}{\sigma\sqrt{T-t}},$

$d_2=d_1-\sigma\sqrt{T-t}$ 

In [1]:
from typing import Tuple
import numpy as np
from scipy.stats import norm

class euro_option:
    def __init__(self,s:float,k:float,r:float,T:int,t:int,mu:float,sigma:float):
        self.s = s # stock
        self.k = k # strike
        self.T = T # strike time
        self.t = t # current time
        self.r = r # interest rate
        self.mu = mu # mu
        self.sigma = sigma # sigma
        self.d1, self.d2 = calc_d1_d2()
        
    def calc_d1_d2(self)->Tuple[float,float]:
        d1=(np.log(self.s/self.k)+(self.r+0.5*self.sigma**2)*(self.T-self.t))/(self.sigma*np.sqrt(self.T-self.t))
        d2=d1-self.sigma*np.sqrt(self.T-self.t) 
        return d1,d2
        
    def get_call(self)->float:
        call=(self.s*norm.cdf(self.d1,0,1)-self.k*np.exp(-self.r*(self.T-self.t))*norm.cdf(self.d2,0,1))
        return call
        
    def get_put(self)->float:
        put=(self.k*np.exp(-self.r*(self.T-self.t))*norm.cdf(-self.d2,0,1)-self.s*norm.cdf(-self.d1,0,1))
        return put

For American Option: we will implement for American Call Option. We will use binary tree:

$s_{t+1}=\left\{\begin{array}{ l l }{as_t}&{\text{ with probability } p}\\{bs_t} & { \text { with probability }}\end{array}\right\},$ 

where $a>1$ and $0<b<1$.

In [15]:
import numpy as np
from typing import TypeVar
def american_option(T:int, p:float, c:float, s0:float, K:float):
    prices = np.zeros((T+1, T+1))
    for i in range(T+1):
        prices[:(i+1),i]=pow(c,(-i + 2*np.arange(i+1)))
    prices = prices*s0
    net_p = prices-K
    net_p*=(net_p>0)  
    val = net_p.copy()
    actions = np.zeros((T+1, T+1))
    for i in range(T-1,-1,-1):
        ex = net_p[:(i+2),i+1].copy()
        tp = net_p[:(i+1),i].copy()
        ex_1 = (1-p)*ex[:-1]+p*ex[1:]
        summ= np.concatenate((ex_1.reshape(len(ex_1),1),tp.reshape(len(tp),1)),1)
        actions[:(i+1),i] = np.apply_along_axis(np.argmax,1,summ)
        val[:(i+1),i] = np.apply_along_axis(max,1,summ)
    return actions, val